<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px"/> 


<!-- # TSO JWebbinar Notebook 1: Downloading and Calibrating `uncal` TSO Products -->
# NIRISS/SOSS Notebook 2: Spectral Extracting and Lightcure Generation for TSO Products
-----

**Authors**:
- **Tyler Baines** | Science Support Analyst | NIRISS Branch | tbaines@stsci.edu
- **Néstor Espinoza** | AURA Assistant Astronomer | Mission Scientist for Exoplanet Science | nespinoza@stsci.edu

**Date Published**: May 1st, 2024

**Last Updated**: May 1st, 2024 

<!-- **Pipeline Version**: 1.12.5 -->

## Table of contents
1. [Introduction](#intro)<br>
   1.1 [Purpose of this Notebook](#purpose)<br>
   1.2 [Input Data](#inputs)<br>
2. [Imports](#imports)<br>
3. [A look at the input data - the `rateints` files](#quick-look)<br>
4. [The JWST Stage 2 Pipeline `Spec2` and our custom steps](#custom)<br>
   4.1 [ Identify and replace bad pixels](#bad)<br>
   4.2 [Background Subtraction](#background_subtraction)<br>
   4.3 [Spectral tracing](#tracing)<br>
   4.4 [Remove 1/f noise from the 2dcutouts](#1overf)<br>
   4.5 [Extract the spectra and obtain wavelength solution](#extraction)<br>
   4.6 [Produce white and spectroscopic light curves](#lightcurve)<br>
5. [Concluding remarks](#remarks)<br>
 

## 1.<font color='white'> </font>Introduction <a class="anchor" id="intro"></a>
<hr style="border:1px solid black">

### 1.1.<font color='white'>-</font>Purpose of this Notebook<a class="anchor" id="purpose"></a> ###

In this notebook, we provide a practical guide for the exploration and generation of NIRISS SOSS light curves and is a continuation of the `01_niriss_soss_detector1_reduction` notebook. We will specifically focus on generating equivalent outputs to the `Spec2` stage of the JWST pipeline, adapting our approach to cater to the unique characteristics of NIRISS/SOSS data. Our process includes methods for spectral tracing and extraction for SOSS observations. The primary objective of this notebook is to fascillitate the production of both white and spectroscopic light curves (as well as fitting products), tailored to the JWST NIRISS/SOSS users for their specific data sets.

### 1.2<font color='white'>-</font>Input Data<a class="anchor" id="inputs"></a> ###

The input data for this notebook, a transit of WASP-39b observed with SOSS CLEAR/GR700XD, are the `Detector1 - RATEINTS` equivalent files produced following the `01_niriss_soss_detector1_reduction` notebook. The full WASP-39b dataset can be downloaded directly from MAST, however, we recommend using the products derived from the first notebook reproducbility purposes. The data set belongs to the <a href="https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1366"> JWST Early Release Science program ERS-1366,</a> which includes 9 groups per integration and 538 integrations per exposure using the `SUBSTRIP256` subarray, covering a total of about 8.25 hours.

## 2. Imports <a class="anchor" id="imports"></a>
<hr style="border:1px solid black">

For this demonstration we will need the following packages to be installed in your python environemnt:
1. numpy 
2. scipy
3. astropy
4. matplotlib
5. jwst
6. pastasoss (see section [4.3](#tracing) for more details about this package)

If you completed the first notebook, you should have a working environment already setup up. If not, you can setup and working environment following these steps. 

```markdown
conda create -n jwst-soss-demo-py3.10 python=3.10 pip
conda activate jwst-soss-demo-py3.10
pip install -r requirements-soss.txt
```

Next we'll import the various python packages that we're actually going to use in this notebook. 

In [ ]:
# ------ General Imports ------
import os
import numpy as np
from scipy import signal
from scipy import ndimage
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import interpolate_replace_nans


# ------ JWST Calibration Pipeline Imports ------
import jwst
from jwst import datamodels

# ------ PASTASOSS import ------
import pastasoss

# ------ Plotting Imports ------
import matplotlib.pyplot as plt

# watermark allows one to see which package versions are being used and 
# some system specs which can be shared easily if problems arise. 
try: 
    %load_ext watermark
    %watermark -v -m -p numpy,astropy,scipy,matplotlib,astroquery,jwst,pastasoss
except:
    print('versions')
    print(f'{np.__name__:<10} : {np.__version__}')
    print(f'{jwst.__name__:<10} : {jwst.__version__}')
    print(f'{pastasoss.__name__:<10} : {pastasoss.__version__}')


Configure plotting parameters.

In [ ]:
plt.rcParams['figure.figsize'] = (12,4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.interpolation'] = 'none'
plt.rcParams['image.cmap'] = 'inferno'

## 3. A look at the input data - the `rateints` files <a class="anchor" id="quick-look"></a>
<hr style="border:1px solid black">

Let's load in the calibrated data product results from the `01_niriss_soss_detector1_reduction` notebook and do a quick inspection again. Then we'll proceed to perform an additional number of steps that would take place during the `Spec2` step such as background subtract and correcting 1/f noise. The `rateints` files contain the slope images for each integration, which is the product we want to use for our analysis. Let's take a look at the contents of the first segment file using the JWST datamodels.

In [ ]:
rateints_data_folder = "data/calibrated/"

files = [
    "jw01366001001_04101_00001-seg001_nis_1_rampfitstep.fits", 
    "jw01366001001_04101_00001-seg002_nis_1_rampfitstep.fits", 
    "jw01366001001_04101_00001-seg003_nis_1_rampfitstep.fits", 
    "jw01366001001_04101_00001-seg004_nis_1_rampfitstep.fits", 
]

rateints = datamodels.open(rateints_data_folder+files[0])

In [ ]:
rateints.info()

In [ ]:
vmin = 0
vmax = 20

sl_fig, axs = plt.subplots(ncols=1, nrows=5, figsize=[18, 17], sharex=True)
for i, ax in enumerate(axs.flat):
    im = ax.imshow(rateints.data[i, :, :], vmin=vmin, vmax=vmax)
    if i == 2:
        ax.set_ylabel('Y-row, pixel', fontsize=15)
    ax.set_title('int = {0}'.format(i+1))

ax.set_xlabel('X-column, pixel', fontsize=15)
sl_fig.subplots_adjust(right=0.95)
cbar_ax = sl_fig.add_axes([0.98, 0.2, 0.02, 0.6])
cbar = sl_fig.colorbar(im, cax=cbar_ax)
cbar.set_label('DN/s', fontsize='x-large')

## 4. The JWST Stage 2 Pipeline `Spec2` and our custom steps <a class="anchor" id="custom"></a>
<hr style="border:1px solid black">

### 4.1 Identify and replace bad pixels <a class="anchor" id="bad"></a>

Looking at these images we can see that each contains a number of bad pixels. We could in theory leave these pixels and continue our analysis. However, when it comes to centroiding and tracing spectra or spectral extraction, it is better to assign some values to these bad pixels. In this step, we will replace these bad pixel. There are many ways to go about doing this step in addition to a step in the pipeline for bad pixel replace. Here we're going to make use of `astropy`'s bad pixel replacement using the `interpolate_replace_nans` method. This this step may take some time to run.

In [ ]:
replace_bad_pixels = True

with_bad_pixels = np.copy(rateints.data[0])

if replace_bad_pixels:
    kernel = Gaussian2DKernel(3, 3)
    for i, image in enumerate(rateints.data):
        rateints.data[i] = interpolate_replace_nans(image, kernel)
else:
    rateints.data = np.nan_to_num(rateints.data) 

In [ ]:
fig, axes = plt.subplots(2, 1, constrained_layout=True)

ax1, ax2 = axes

ax1.set_title('Int = 1')
ax1.set_xlabel('x [pixel]')
ax1.set_ylabel('y [pixel]')
ax1.imshow(with_bad_pixels, vmin=0, vmax=20)
fig.colorbar(ax=ax1, label='DN/s')

ax2.set_title('Int = 1, w/ Bad Pixel Replacement')
ax2.set_xlabel('x [pixel]')
ax2.set_ylabel('y [pixel]')
ax2.imshow(rateints.data[0], vmin=0, vmax=20)
fig.colorbar(ax=ax2, label='DN/s')

plt.show()

Great, the image looks much cleaner now, and our interpolated replacements are likely to be close to the true values. 

## 4.2 Background Subtraction <a class="anchor" id="background_subtraction"></a>

Here, we will perform a background subtraction using a model/template of the dispersed zodiacal background which we can estimate a scaling factor to scale the template by and subtract from the science frame to remove the background structure. The SOSS background model can be found and downloaded [here](https://stsci.app.box.com/s/9qohomufvzf84tn4gjawnko11oruq2jw) as it will be required for the next steps.

<div class="alert alert-block alert-info"><b>Notes:</b> 
JWST pipeline does not automaticallyy perform background subtraction at this time. It's been observed that the background changes from target-to-target thus to remove the background user are encourage to use a background template model which can be scaled and removed from the science frames. 
</div>


Lets, load in the background model and plot the template. 


In [ ]:
# path to where a user stores the file. 
bkg_model_path = "~/Downloads/model_background256.npy"
bkg_model_path = os.path.expanduser(bkg_model_path)

bkg_template = np.load(bkg_model_path)

plt.figure(figsize=(12,8))

plt.subplot(3,1,1)
plt.imshow(np.nan_to_num(rateints.data[0]), vmin=0, vmax=20)
plt.title('SOSS Data')
plt.ylabel('y [pixel]')
plt.xlabel('x [pixel]')

plt.subplot(3,1,2)
plt.imshow(bkg_template)
plt.title('SOSS Sky Background Tempalte')
plt.ylabel('y [pixel]')
plt.xlabel('x [pixel]')

plt.subplot(3,1,3)
plt.title('Horizontal Cross Section: Row 50')
plt.plot(bkg_template[50])
plt.xlabel('x [pixel]')
plt.ylabel('DN/s [pixel]')

plt.tight_layout()
plt.show()

Next, were going to define and run a helper function with the intention of estimating the scale factor to scale the template by. This method estimates the scale factor by looking a small cutout region free of the dispersed spectra and centered around where the background intensity jumps. Here we'll extract the subarray between pixel rows 210 to 250, and pixel columns 500 to 800. 

In [ ]:
def get_background_scaler(image, bkg_model, box_bounds=[210,250,600,800], percentile_range=[0.25, 0.75], plot=False):

    # extract subregion of background
    iy1, iy2, ix1, ix2 = box_bounds
    bkg_postage = image[iy1:iy2, ix1:ix2]
    model_bkg_postage = bkg_model[iy1:iy2, ix1:ix2]

    # mask the bad pixel and compute ratios
    bad_pixels = np.isnan(bkg_postage)
    ratio = bkg_postage[~bad_pixels] / model_bkg_postage[~bad_pixels]
    ratio = ratio.flatten()

    # estiamte the scale factor i.e. the median ratio 
    idx_sorted = np.argsort(ratio)
    npixels = len(ratio)

    lower_percentile, upper_percentile = percentile_range

    pixel_index = np.arange(len(ratio))

    idx_lower = int(npixels*lower_percentile)
    idx_upper = int(npixels*upper_percentile)

    median_ratio = np.median(ratio[idx_sorted][idx_lower:idx_upper])

    if plot:
        print('plotting scaling results')
        original_pixels = bkg_postage.flatten()
        bkg_substracted_pixels = bkg_postage.flatten() - model_bkg_postage.flatten() * median_ratio
        non_outliers_original = np.where( (bkg_postage.flatten() < 10.5)&(bkg_postage.flatten() > 1.0))[0]
        # Plotting
        fig, axes = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
        fontsize = 16

        # Plot 1: Background Distrubition ratio
        # axes[0].set_title('Background Distrubition')
        axes[0].plot(pixel_index, ratio[idx_sorted], label='Background Distribution')
        axes[0].plot(pixel_index[idx_lower:idx_upper], ratio[idx_sorted][idx_lower:idx_upper], label=f'{lower_percentile*100} to {upper_percentile*100} Percentile Region')
        axes[0].set_xlabel('Pixel index', fontsize=fontsize)
        axes[0].set_ylabel('Data / Model', fontsize=fontsize)
        axes[0].tick_params(axis='both', labelsize=fontsize)
        axes[0].legend()
        
        # Plot 2: Ratio distribution
        axes[1].plot(ratio, '.', label='Ratio')
        axes[1].plot([0, len(ratio)], [median_ratio, median_ratio], label='Median Ratio')
        axes[1].set_xlabel('Pixel index', fontsize=fontsize)
        axes[1].set_ylabel('Data / Model', fontsize=fontsize)
        axes[1].tick_params(axis='both', labelsize=fontsize)
        axes[1].legend()

        # Plot 3: Original vs. Corrected Pixels
        axes[2].plot(original_pixels[non_outliers_original], label='Original')
        axes[2].plot(bkg_substracted_pixels[non_outliers_original], label='Corrected')
        axes[2].plot([0,8000], [0,0], c='k')
        axes[2].set_xlabel('Pixel Index', fontsize=fontsize)
        axes[2].set_ylabel('Pixel Value', fontsize=fontsize)
        axes[2].tick_params(axis='both', labelsize=fontsize)
        axes[2].legend()

        plt.tight_layout()
        plt.show()
    
    return median_ratio

In [ ]:
# for some diagnostic set plot=True
median_ratio = get_background_scaler(rateints.data[0], bkg_template, plot=True)

With this method we do a good job removing the background counts by subtracting off the scaled background template, and the pixel values are approximately centered around zero. 

Lets look at a cross section to compare the before and after Background Subtraction using the template.

In [ ]:
plt.subplot(2,1,1)
plt.title('Background Subtraction')
plt.plot(rateints.data[0, 25])
plt.plot((bkg_template * median_ratio)[25])
plt.ylabel('DU/s')
plt.ylim(2, 20)

plt.subplot(2,1,2)
plt.plot(rateints.data[0, 25])
plt.plot(rateints.data[0, 25] - (bkg_template * median_ratio)[25])
plt.xlabel('x [pixel]')
plt.ylabel('DU/s')
plt.ylim(-2, 20)
plt.show()

Let's see how well we do to remove the background from the actual images. 

In [ ]:
bkg_corrected_frames = rateints.data - bkg_template * median_ratio

plt.figure(figsize=(15,10))
plt.subplot(211)
plt.title('Before Background Subtraction')
plt.imshow(np.log1p(np.nan_to_num(rateints.data[0])), vmin=-1, vmax=3)
plt.colorbar()
plt.subplot(212)
plt.title('After Background Subtraction')
plt.imshow(np.log1p(np.nan_to_num(bkg_corrected_frames[0])), vmin=-1.5, vmax=3)
plt.colorbar()
plt.show()

This looks pretty good. Our method to remove the background by scaling a templace of the background structure is effectively removed from the science image. 

### 4.3 Spectral Tracing <a class="anchor" id="tracing"></a>

To trace the spectrum one would need to apply methods such as centroiding, cross correlation, modeling, and so on. Furthermore, SOSS observations are susceptible to rotations and shifts of the positions of dispersed spectral orders on the the detector which is dependent on the position of the pupil wheel that houses the GR700XD which slightly overshoots/undershoots its commanded position. As a result, this impacts the spectral trace positions and wavelength solution observation to observation. To remedy this, a tool was recently released called [PASTASOSS](https://github.com/spacetelescope/pastasoss) (Predicting Accurate Spectral Traces for Astrophysical SOSS Spectra) to predict the spectral trace positions for any niriss SOSS observations given its PWCPOS value. We will be using PASTASOSS to acquire the trace positions for the spectral orders 1 and 2.

To use PASTASOSS and get the spectral traces for orders 1 and 2, we need to extract the pupil wheel position angle PWCPOS from the header information. We can find this information by doing a search on the datamodel using the keyword `pupil_position`. Once we know the value we can use the `get_soss_traces()` method in the PASTASOSS package to predict the spectral traces positions and their corresponding wavelength values. At this time, the software fully supports order 1, partial support for order 2 and will support order 3 in the future. 

<!-- <div class="alert alert-block alert-info">
<b>Tip:</b> PASTASOSS supports order 1 fully and partially for order 2.  
</div> -->

In [ ]:
rateints.search('pupil_position')

In [ ]:
pwcpos = rateints.meta.instrument.pupil_position

traces_order1, traces_order2, _ = pastasoss.get_soss_traces(pwcpos, order="123", interp=True)

print(f"PWCPOS: {pwcpos} deg")
print(traces_order1)
print(traces_order2)

PASTASOSS outputs a `TraceModel` object which encapsulate the spectral trace infomation for a given order. 

Let's plot the data with traces positions for the corresponding pupil position.

In [ ]:
plt.figure()
plt.imshow(np.nan_to_num(bkg_corrected_frames[0]), vmin=0, vmax=10)
plt.plot(traces_order1.x, traces_order1.y, color='cornflowerblue', lw=3, label='Order 1')
plt.plot(traces_order2.x, traces_order2.y, color='orangered', lw=3, label='Order 2')
plt.ylabel('y [pixel]')
plt.xlabel('x [pixel]')
plt.legend()
plt.show()

That looks pretty good! One could measure the spectral traces and perform their own wavelength calibration, however, by using PASTASOSS we can quickly predict the postions and wavelength with sub-pixel accuracy for any NIRISS/SOSS observations. 

<div class="alert alert-block alert-info"><b>Notes:</b> 
PASTASOSS will provide full support for order 2 providing a full trace and improved wavelength solution in a future release along with an update to this notebook.
</div>

### 4.4 Remove 1/f noise from the 2D cutouts <a class="anchor" id="1overf"></a>

JWST detector readout electronics (aka SIDECAR ASICs) generate significant 1/f noise during detector operations and signal digitization. When using NIRISS, this 1/f noise appears as vertical banding that spans the entire width of the 2D spectral image, and varies from column to column. If not handled properly, the 1/f noise can introduce systematic errors and extra scatter in the light curves. For more information, please visit: <a href="https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-time-series-observations/jwst-time-series-observations-noise-sources#JWSTTimeSeriesObservationsNoiseSources-1/fnoise">JWST Time-Series Observations Noise Sources.</a>

One way to estimate and remove 1/f noise for the SOSS data is to measure the median count level of a column using pixels outside of the Point Spread Function (PSF). This median flux is then subtracted from the entire column. The function below performs this task for a given image using background region above and below the spectrum using the traces we just defined. 

We'll begin by defining a helper function to perform the 1/f noise removal for a single segmented datast. 


In [ ]:
def correct_1f(median_frame, frame, x, y, min_bkg=25, max_bkg=35, mask=None, 
               scale_factor=1., return_1f = False):
    """Method to perform 1/f noise removal that performs a median integration
    frame subtraction to remove the signal to isolate and remove 1/f noise.""" 
    new_frame = np.copy(frame)
    ms = frame - median_frame * scale_factor

    if return_1f:
        one_over_f = np.zeros(len(x))
    
    # Go column-by-column substracting values around the trace:
    for i in range(len(x)):
        column = int(x[i])
        row = int(y[i])
        min_row = np.max([0, row - max_bkg])
        max_row = np.min([256, row + max_bkg])
        bkg = np.append(ms[min_row:row-min_bkg, column], ms[row+min_bkg:max_row, column])
        new_frame[:, column] = new_frame[:, column] - np.nanmedian(bkg)
        
        if return_1f:
            one_over_f[i] = np.nanmedian(bkg)
        
    if return_1f:
        return one_over_f, new_frame 
    else:
        return new_frame

Here, we can check if the a transit is present, however, since we are only looking at the first segmented dataset we will observe the relative flux of the out-of-tranist. This is import for our 1/f correction step as we'll need to scale the median frame to remove the underlining signal to better isolate the 1/f noise. This step will be repeat in a later section when we use all the data and see the entire tranist lightcurve. 

In [ ]:
postage = rateints.data[:, 20:60, 1500:1550]
timeseries = np.sum(postage, axis = (1,2))

# Create smoothed version to use for background + 1/f corrections:
smoothed_petit_transit = signal.medfilt(timeseries / np.median(timeseries[0:80]), 11)

plt.figure(figsize=(10,3.5))
plt.plot(timeseries / np.median(timeseries[0:80]), '.', color = 'orangered')
plt.plot(smoothed_petit_transit, '-', color = 'cornflowerblue')

plt.xlim(0, len(timeseries))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Relative flux', fontsize = 18)
plt.xlabel('Integration number', fontsize = 18)
plt.show()

Now lets run the function and see how the data change. 

In [ ]:
bkg_corrected_median_oot = np.median(bkg_corrected_frames, axis=0)

# remove 1/f noise and scale median frame using relative flux
one_over_f, new_frame = correct_1f(bkg_corrected_median_oot, 
                                   bkg_corrected_frames[0], 
                                   traces_order1.x,
                                   traces_order1.y, 
                                   scale_factor = smoothed_petit_transit[0], 
                                   return_1f = True)

plt.figure(figsize=(12,8))

plt.subplot(411)
plt.title('SCI: UNCORRECTED')
im = plt.imshow(rateints.data[0])
im.set_clim(-1,10)

plt.subplot(412)
plt.title('SCI: BACKGROUND REMOVED')
im = plt.imshow(bkg_corrected_frames[0])
im.set_clim(-1,3)

plt.subplot(413)
plt.title('SCI: BACKGROUND + 1/f REMOVED')
im = plt.imshow(new_frame)
im.set_clim(-1,3)

plt.subplot(414)
plt.title('1/f NOISE')
im = plt.imshow(bkg_corrected_frames[0] - new_frame)
im.set_clim(-1,1)

plt.tight_layout()
plt.show()

As you can see this is a much more subtle effect than the background, however, the 1/f correction seems to have done a decent job at identifying the banding across the columns. The impact of 1/f noise is likely to change from dataset to dataset, particularly as a function of the number of groups within your integrations, or how heavily the detector is exposed. 

Potential improvements could be made with more sophisticated techniques, such as: performing 1/f subtraction at the group level prior to the generation of the *rateints.fits files, using a more explicit mask to identify unilluminated pixels, or performing a polynomial fit rather than using the median estimate of the 1/f in each column. However, we do not explore such techniques for this tutorial.

### 4.5 Spectral Extraction <a class="anchor" id="extraction"></a>

Now that we have obtain the spectral traces and cleaned the data, we can proceed with the spectral extraction. For this task, we will use a simple aperture extraction to acquire the order 1 and order 2 spectra. However, the JWST pipeline uses [ATOCA](https://ui.adsabs.harvard.edu/abs/2022PASP..134i4502D/abstract) a complex spectral extraction algorithm specifically developed for NIRISS/SOSS data. ATOCA is a complex algorithm that enables simultaneously extraction of the spectral orders as well as disentangling signal from the order 1 and 2 which overlap slightly over a short range. 

In [ ]:
def simple_extraction(image, x, y, aperture_radius, background_radius=50, error_image=None, correct_bkg=True, method = 'sum'):
    """
    This function takes as inputs two arrays (x,y) that follow the trace,
    and returns the added flux over the defined aperture radius (and its error, if an error image
    is given as well), substracting in the way any background between the aperture radius and the
    background radius. The background is calculated by taking the median of the points between the
    aperture_radius and the background_radius.

    Parameters
    ----------
    image: ndarray
        Image from which the spectrum wants to be extracted
    x: ndarray
        Array with the x-axis of the trace (i.e., the columns, wavelength direction)
    y: ndarray
        Array with the y-axis of the trace (i.e., rows, spatial direction)
    aperture_radius: float
        Distance from the center of the trace at which you want to add fluxes.
    background_radius: float
        Distance from the center of the trace from which you want to calculate the background. The
        background region will be between this radius and the aperture_radius.
    error_image: ndarray
        Image with the errors of each pixel value on the image ndarray above
    correct_bkg: boolean
        If True, apply background correction. If false, ommit this.
    method : string
        Method used to perform the extraction. Default is `sum`; `average` takes the average of the non-fractional pixels 
        used to extract the spectrum. This latter one is useful if the input is a wavelength map.
    """

    method = method.lower()

    # If average method being used, remove background correction:
    if method == 'average':
        correct_bkg = False

    # Create array that will save our fluxes:
    flux = np.zeros(len(x))
    if error_image is not None:
        flux_error = np.zeros(len(x))
    max_column = image.shape[0] - 1

    for i in range(len(x)):

        # Cut the column with which we'll be working with:
        column = image[:,int(x[i])]
        if error_image is not None:
            variance_column = error_image[:,int(x[i])]**2

        # Define limits given by the aperture_radius and background_radius variables:
        if correct_bkg:
            left_side_bkg = np.max([y[i] - background_radius, 0])
            right_side_bkg = np.min([max_column, y[i] + background_radius])
        left_side_ap = np.max([y[i] - aperture_radius, 0])
        right_side_ap = np.min([max_column, y[i] + aperture_radius])

        # Extract background, being careful with edges:
        if correct_bkg:
            bkg_left = column[np.max([0, int(left_side_bkg)]) : np.max([0, int(left_side_ap)])]
            bkg_right = column[np.min([int(right_side_ap), max_column]) : np.max([int(right_side_bkg), max_column])]
            bkg = np.median(np.append(bkg_left, bkg_right))
        else:
            bkg = 0.

        # Substract it from the column:
        column -= bkg

        # Perform aperture extraction of the background-substracted column, being careful with pixelization
        # at the edges. First, deal with left (up) side:
        l_decimal, l_integer = np.modf(left_side_ap)
        l_integer = int(l_integer)
        if l_decimal < 0.5:
            l_fraction = (0.5 - l_decimal) * column[np.min([l_integer, max_column])]
            l_limit = l_integer + 1
            if error_image is not None:
                l_fraction_variance = ((0.5 - l_decimal)**2) * variance_column[np.min([l_integer, max_column])]
        else:
            l_fraction = (1. - (l_decimal - 0.5)) * column[np.min([l_integer + 1, max_column])]
            l_limit = l_integer + 2
            if error_image is not None:
                l_fraction_variance = ((1. - (l_decimal - 0.5))**2) * variance_column[np.min([l_integer + 1, max_column])]

        # Now right (down) side:
        r_decimal, r_integer = np.modf(right_side_ap)
        r_integer = int(r_integer)
        if r_decimal < 0.5:
            r_fraction = (1. - (0.5 - r_decimal)) * column[np.min([max_column, r_integer])]
            r_limit = r_integer
            if error_image is not None:
                r_fraction_variance = ((1. - (0.5 - r_decimal))**2) * variance_column[np.min([max_column, r_integer])]
        else:
            r_fraction = (r_decimal - 0.5) * column[np.min([max_column, r_integer + 1])]
            r_limit = r_integer + 1
            if error_image is not None:
                r_fraction_variance = ((r_decimal - 0.5)**2) * variance_column[np.min([max_column, r_integer + 1])]

        # Save total flux in current column:
        if method == 'sum':
            flux[i] = l_fraction + r_fraction + np.sum(column[l_limit:r_limit])

        elif method == 'average':
            flux[i] = np.mean(column[l_limit:r_limit])

        else:
            raise Exception('Method "'+method+'" currently not supported for aperture extraction. Select either "sum" or "average".')

        if error_image is not None:
            # Total error is the sum of the variances:
            flux_error[i] = np.sqrt(np.sum(variance_column[l_limit:r_limit]) + l_fraction_variance + \
                                    r_fraction_variance)
    if error_image is not None:
        return flux, flux_error
    else:
        return flux

Lets compare the spectra before and after the background correction step

In [ ]:
spectrum_order1 = simple_extraction(np.nan_to_num(rateints.data[0]), traces_order1.x, traces_order1.y, aperture_radius=15)
spectrum_order2 = simple_extraction(np.nan_to_num(rateints.data[0]), traces_order2.x, traces_order2.y, aperture_radius=15)

spectrum_order1_clean = simple_extraction(new_frame, traces_order1.x, traces_order1.y, aperture_radius=15)
spectrum_order2_clean = simple_extraction(new_frame, traces_order2.x, traces_order2.y, aperture_radius=15)

plt.figure(figsize=(12,10))

# plot order 1 spectra
plt.subplot(311)
plt.title('Order 1')
plt.xlabel('Wavelength [$\mu m$]')
plt.ylabel('ADU/s')
plt.plot(traces_order1.wavelength, spectrum_order1, label='Before')
plt.plot(traces_order1.wavelength, spectrum_order1_clean, label='After')
plt.legend()

# plot order 2 spectra
plt.subplot(312)
plt.title('Order 2')
plt.xlabel('Wavelength [$\mu m$]')
plt.ylabel('ADU/s')
plt.plot(traces_order2.wavelength, spectrum_order2, label='Before')
plt.plot(traces_order2.wavelength, spectrum_order2_clean, label='After')

# plot difference or ratio
plt.subplot(313)
plt.xlabel('Wavelength [$\mu m$]')
plt.ylabel('Difference')
plt.plot(traces_order1.wavelength, spectrum_order1-spectrum_order1_clean, 'b', label='Order 1')
plt.legend(loc='upper left')
plt.twiny()
plt.plot(traces_order2.wavelength, spectrum_order2-spectrum_order2_clean, 'r', label='Order 2')
plt.legend(loc="upper right")
plt.ylim(-2, 5)
plt.tight_layout()
plt.show()


This looks good. There's not much of a significant change in the spectra, however, some underlying structure has been removed.

Also notice the 'bumps' in order 2, this is the result of contamination from the zeroth order of field stars, which can also be seen in the 2D images. We do not perform any special treatment for these zero orders here. 

### 4.6 Generating Lightcurves  <a class="anchor" id="lightcurve"></a>

Now lets put everything we've done so far together and generate some lightcurves. So far we've just been using the first segment of our data files, so lets now load in all the data and rerun what we've done. 

In [ ]:
tso_images = []
err_images = []
all_times = []
all_pwcpos = []

for file in files:  
    print(file.split('/')[-1])
    # load the file and extract relavent data 
    with datamodels.open(rateints_data_folder + file) as tso_data:
        data = tso_data.data
        err = tso_data.err
        pwcpos = tso_data.meta.instrument.pupil_position
        times = tso_data.int_times.int_mid_BJD_TDB

        tso_images.append(data)
        err_images.append(err)
        all_pwcpos.append(pwcpos)
        all_times.append(times)

tso_images = np.vstack(tso_images)
err_images = np.vstack(err_images)
all_times = np.hstack(all_times)
all_pwcpos = np.array(all_pwcpos)

In [ ]:
tso_images.shape, all_times.shape, all_pwcpos.shape, err_images.shape

Replace the bad pixel in all the images. This may take some time  ~ 5-min.

In [ ]:
replace_bad_pixels = False

if replace_bad_pixels:
    kernel = Gaussian2DKernel(3, 3)
    for i, image in enumerate(tso_images):
        tso_images[i] = interpolate_replace_nans(image, kernel)
else:
    tso_images = np.nan_to_num(tso_images) 

In [ ]:
print(f"Checking PWCPOS is consistent around files: {np.unique(all_pwcpos)}")

Grab the traces once again using PASTASOSS.

In [ ]:
pwcpos = np.unique(all_pwcpos)[0]

traces_order1, traces_order2, _ = pastasoss.get_soss_traces(pwcpos, interp=True)

Now we can get a quick glimpse of what our light curve looks like before any sort of background or 1/f correction. 

In [ ]:
postage = np.nan_to_num(tso_images[:, 20:60, 1500:1550])
timeseries = np.sum(postage, axis = (1,2))
smoothed_petit_transit = ndimage.median_filter(timeseries / np.median(timeseries[0:80]), 11)

plt.figure(figsize=(10,3.5))
plt.plot(timeseries / np.median(timeseries[0:80]), '.', color = 'orangered')
plt.plot(smoothed_petit_transit, '-', color = 'cornflowerblue')
plt.xlim(0, len(timeseries))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
    
plt.ylabel('Relative flux', fontsize = 18)
plt.xlabel('Integration number', fontsize = 18)
plt.show()

Let's calculate a median background corrected image for the out of transit data to be used as our reference for the 1/f correction.. 

In [ ]:
# Replace here integrations before and after transit:
int_before_transit = 200
int_after_transit = 410

bkg_model = bkg_template * median_ratio
bkg_corrected_oot_integrations = np.vstack((
    tso_images[:int_before_transit, :, :] - bkg_model, 
    tso_images[int_after_transit:, :, :] - bkg_model)
    )

bkg_corrected_median_oot = np.nanmedian(bkg_corrected_oot_integrations, axis = 0 )

Then we can perform the 1/f correction and extraction across all of the images. This step might take some time ~2 min. 

In [ ]:
# spectral extraction box size
aperture_radius = 15

# store results
spectra_order1 = []
spectra_order2 = []
errs_order1 = []
errs_order2 = []

for image, err, scale in zip(np.nan_to_num(tso_images), err_images, smoothed_petit_transit):
    # median_ratio = get_background_scaler(image, bkg_template, plot=False)
    bkg_corrected_frame = image - bkg_model
    one_over_f, new_frame = correct_1f(bkg_corrected_median_oot, 
                                       bkg_corrected_frame, 
                                       traces_order1.x,
                                       traces_order1.y, 
                                       scale_factor = scale, 
                                       return_1f = True)
    
    # extract order 1 and order 2 spectra from cleaned images
    spectrum_order1, spectrum_err1 = simple_extraction(new_frame, traces_order1.x, traces_order1.y, aperture_radius=15, error_image=err)
    spectrum_order2, spectrum_err2 = simple_extraction(new_frame, traces_order2.x, traces_order2.y, aperture_radius=15, error_image=err)

    # append results
    spectra_order1.append(spectrum_order1)
    spectra_order2.append(spectrum_order2)
    errs_order1.append(spectrum_err1)
    errs_order2.append(spectrum_err2)

In [ ]:
# Quickly convert things to arrays where needed. 
integration_index = np.arange(len(tso_images))
spectra_order1 = np.asarray(spectra_order1)
spectra_order2  = np.asarray(spectra_order2)
errs_order1  = np.asarray(errs_order1)
errs_order2  = np.asarray(errs_order2)

Now we can plot the collection of extracted 1D spectra for each integration on top of each other. As you can see we have an almost bimodal distribution, where the spectra with more flux are those that are out-of-transit, the spectra with less flux are in-transit, and the smaller number inbetween trace the flux variations at ingress and egress. 

In [ ]:
plt.figure(figsize=(12,8))

# plot order 1 spectra
plt.subplot(3,1,1)
plt.title('Order1 Close-up: In- and Out-of Transit Spectra')
plt.xlabel("wavelength [$\mu m$]")
plt.ylabel("DN/s")
for spec in spectra_order1:
    plt.plot(traces_order1.wavelength,  spec, 'k', alpha=0.05)

plt.xlim(1.1, 1.2)
plt.ylim(7500, 8250)

# plot order 2 spectra
plt.subplot(3,1,2)
plt.title('Order 2 Close-up: In- and Out-of Transit Spectra')
plt.xlabel("wavelength [$\mu m$]")
plt.ylabel("DN/s")
for spec in spectra_order2:
    plt.plot(traces_order2.wavelength,  spec, 'k', alpha=0.05)
plt.xlim(0.7, 0.74)
plt.ylim(3200, 3800)

plt.tight_layout()
plt.show()

This looks pretty good. As we see removing the background at the integration level is pretty stable. 

We can also plot a 2D representation of this variation, as seen below. 

In [ ]:
lightcurve_order1 = (spectra_order1 / np.nanmedian(spectra_order1, 0) - 1) * 1e6 
lightcurve_order2 = (spectra_order2 / np.nanmedian(spectra_order2, 0) - 1) * 1e6 

# plot the results
n_integrations = np.arange(len(lightcurve_order1))

vmax = 20000
vmin = -15000

plt.figure(figsize=(12,8), dpi=187)
plt.subplot(2,1,1)
plt.title('WASP-39 Order 1 Lightcurves')
plt.pcolormesh(traces_order1.wavelength, 
               n_integrations, 
               lightcurve_order1, vmin=vmin, vmax=vmax)
plt.xlabel("Wavelength [$\mu m$]")
plt.ylabel("Integration")

cbar = plt.colorbar()
cbar.ax.set_ylabel('ppm', rotation=270, fontsize = 15)
cbar.ax.get_yaxis().labelpad = 15

plt.subplot(2,1,2)
plt.title('WASP-39 Order 2 Lightcurves')
plt.pcolormesh(traces_order2.wavelength, 
               n_integrations, 
               lightcurve_order2, vmin=vmin, vmax=vmax)
plt.xlabel("Wavelength [$\mu m$]")
plt.ylabel("Integration")

cbar = plt.colorbar()
cbar.ax.set_ylabel('ppm', rotation=270, fontsize = 15)
cbar.ax.get_yaxis().labelpad = 15

plt.tight_layout()
plt.show()

As you can see, there is a clear band where the drops across all wavelengths due to the transit. By taking vertical slices across this plot, we can in turn look at the light curves at a wavelength of interest. For example: 

In [ ]:
all_times_hr = (all_times-np.nanmean(all_times))*24.

# temp plot of the lightcurves nestor needs to review this notebook. 
plot_wavelengths = traces_order1.wavelength[10:-10:500]

plt.figure(figsize=(12, 5.5))

offset = 40000
for wavelength in plot_wavelengths[::-1]:
    plt.plot(all_times_hr, lightcurve_order1[:, np.where(traces_order1.wavelength==wavelength)[0]]+offset, label=f'$\lambda={wavelength:.2f} \\ \mu m$')
    offset -= 20000
# plt.plot(all_times_hr, lightcurve_order1[:, 100], color='red', marker="o", markersize=2)
# plt.plot(all_times_hr, lightcurve_order1[:, 1100], color='green', marker="o", markersize=2)
# plt.plot(all_times_hr, lightcurve_order1[:, 2000], color='blue', marker="o", markersize=2)
plt.legend(loc='lower right', ncols=2)
plt.xlabel('Time since mid-exposure, hr', fontsize=15)
plt.ylabel('Normalized flux', fontsize=15)


plt.show()

Lets look at the out-of-transit flux standard deviation as a function of wavelength. 

In [ ]:
lightcurve_order1_std = np.vstack((
    lightcurve_order1[:int_before_transit,], 
    lightcurve_order1[int_after_transit:])
    ).std(0)

lightcurve_order2_std = np.vstack((
    lightcurve_order2[:int_before_transit,], 
    lightcurve_order2[int_after_transit:])
    ).std(0)

fig, ax = plt.subplots(2,1, figsize=(12,6))#

ax1= ax[0]
ax1.set_title('Order 1')
ax1.plot(traces_order1.wavelength, lightcurve_order1_std)
ax1.set_ylabel("out-of-transit stddev")

ax2= ax[1]
ax2.set_title('Order 2')
ax2.plot(traces_order2.wavelength, lightcurve_order2_std)
ax2.set_ylabel("out-of-transit stddev")
ax2.set_xlabel(r"wavelength ($\mu m$)")

plt.tight_layout()
# plt.legend()
plt.show()

### 5. Concluding remarks<a class="anchor" id="remarks"></a>

 <!--need to update the special thanks sections.   -->

**Need to update**

In this notebook, we demonstrated how to obtain light curves and fitting data products starting from the rateints files, and using a step from the JWST pipeline and our custom steps. The saved data products can now be provided to light curve fitting codes for measurements of the physical properties of the exoplanet and obtaining a transmission spectrum. It should be pointed out that the analyses performed here are only a subset of the possible analyses one can perform, and are in no way the final word on _how_ JWST data _should_ be analyzed. This will be solidified more and more as data comes and best practices are established in the current and future cycles.

In conclusion, I would like to express my gratitude to the JWST NIRISS team that has supported the creation of this notebook through discussions and testing, which have improved the notebook. In particular, special thanks to the Time-Series Observations Working Group at STScI, including Néstor Espinoza, Leonardo Ubeda, Sarah Kendrew, Elena Manjavacas, Brian Brooks, Mike Reagan, Loïc Albert, Everett Schlawin, Stephan Birkmann among others. To the NIRCam IDT team for multiple fruitful discussions, including Everett Schlawin, Thomas Beatty, Tom Greene and Jarron Leisenring. To the ERS Transiting Exoplanet team who have provided several venues for discussion and community input. To the several JWST team members, behind the pipeline and the mission itself, including and in no particular order Bryan Hilbert, Armin Rest, Anton Koekemoer, Alicia Canipe, Melanie Clarke, James Muzerolle, Kayli Glidic, Jeff Valenti and Karl Gordon.